# 🧠 Agent Memory Search: Personalized Banking Assistant

Welcome to our **Agent Memory Search** tutorial! This notebook demonstrates how to build an AI banking assistant that **remembers customer preferences** across conversations using the **Memory Search Tool**.

## What You'll Build

A **Personalized Banking Advisor** that:
- 🧠 **Remembers** customer preferences (risk tolerance, account types, communication preferences)
- 💬 **Recalls** past interactions in new conversations
- 🏦 **Provides personalized** financial guidance based on stored memories
- 📊 **Maintains context** across multiple banking sessions

## Why Memory Matters in Banking

| Without Memory | With Memory |
|----------------|-------------|
| Customer repeats preferences every call | Agent recalls "You prefer low-risk investments" |
| Generic product recommendations | Personalized suggestions based on history |
| No context from previous sessions | Seamless continuation of past discussions |
| Impersonal experience | "Welcome back! Last time we discussed..." |

## Architecture Overview

```
┌──────────────────────────────────────────────────────────────────────┐
│                    MEMORY-ENABLED BANKING AGENT                       │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│   Conversation 1                    Conversation 2                    │
│   ┌─────────────┐                  ┌─────────────┐                   │
│   │ "I prefer   │                  │ "What should│                   │
│   │  low-risk   │                  │  I invest   │                   │
│   │  investments"│                 │  in?"       │                   │
│   └──────┬──────┘                  └──────┬──────┘                   │
│          │                                │                          │
│          ▼                                ▼                          │
│   ┌─────────────┐                  ┌─────────────┐                   │
│   │   Memory    │◄────────────────►│   Memory    │                   │
│   │   Store     │   Automatic      │   Search    │                   │
│   │  (Extract)  │   Retrieval      │  (Recall)   │                   │
│   └──────┬──────┘                  └──────┬──────┘                   │
│          │                                │                          │
│          ▼                                ▼                          │
│   ┌──────────────────────────────────────────────┐                   │
│   │           Azure AI Memory Store               │                   │
│   │  • User Profile (preferences, risk tolerance) │                   │
│   │  • Chat Summary (conversation highlights)     │                   │
│   │  • Semantic Search (find relevant memories)   │                   │
│   └──────────────────────────────────────────────┘                   │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

## Prerequisites

- Microsoft Foundry project with appropriate permissions
- **Chat model deployment** (e.g., `gpt-4o`)
- **Embedding model deployment** (e.g., `text-embedding-3-large`)
- A `.env` file in the project root containing (uses existing variables):
  ```bash
  AI_FOUNDRY_PROJECT_ENDPOINT=<your-ai-foundry-project-endpoint>
  AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
  EMBEDDING_MODEL_DEPLOYMENT_NAME=text-embedding-3-large
  TENANT_ID=<your-azure-tenant-id>
  ```

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Memory Store** | Persistent storage for extracted memories from conversations |
| **Memory Search Tool** | Agent tool that retrieves relevant memories during conversations |
| **Scope** | User identifier to isolate memories per customer |
| **User Profile** | Automatically extracted preferences and traits |
| **Chat Summary** | Condensed highlights from past conversations |

### ⚠️ Important Financial Disclaimer
> **The financial information provided by this notebook is for general educational and demonstration purposes only.** Always consult with qualified financial advisors before making financial decisions.

📚 **Reference**: [Azure AI Agent Memory Documentation](https://learn.microsoft.com/azure/ai-services/agents/concepts/memory)

---

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI:

```bash
az login --use-device-code
```

This provides device code authentication, useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

---

## 📦 Step 1: Install Required Packages

The required packages should already be installed via the `requirements.txt` file at the project root:

```bash
pip install -r requirements.txt
```

If packages are not installed, uncomment and run the cell below.

In [ ]:
# Packages should be installed via requirements.txt at project root
# If not installed, uncomment the line below and run this cell:

# %pip install azure-ai-projects>=2.0.0b1 azure-identity openai python-dotenv --quiet

print("✅ Packages ready (installed via requirements.txt)")

---

## 🔧 Step 2: Load Configuration and Initialize Clients

Load environment variables and establish connections to Microsoft Foundry.

In [ ]:
import os
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    MemoryStoreDefaultDefinition,
    MemorySearchTool,
    PromptAgentDefinition,
    MemoryStoreDefaultOptions,
)

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get configuration - using existing .env variables
tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")
embedding_deployment = os.environ.get("EMBEDDING_MODEL_DEPLOYMENT_NAME", "text-embedding-3-large")

# Memory store uses the same models as the rest of the project
memory_chat_model = model_deployment
memory_embedding_model = embedding_deployment

print("🧠 Agent Memory Search - Personalized Banking Assistant")
print("=" * 60)
print(f"🔑 Tenant ID: {tenant_id}")
print(f"📍 Project Endpoint: {project_endpoint[:50]}..." if project_endpoint else "❌ Missing")
print(f"🤖 Agent Model: {model_deployment}")
print(f"🧠 Memory Chat Model: {memory_chat_model}")
print(f"📊 Memory Embedding Model: {memory_embedding_model}")

In [ ]:
# Initialize AIProjectClient with Azure CLI authentication
print("🔐 Initializing clients with AzureCliCredential...")
print("   Make sure you've run 'az login' in your terminal first!")
print("")

credential = AzureCliCredential(tenant_id=tenant_id)

# Create the project client
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

# Get the OpenAI client for conversations
openai_client = project_client.get_openai_client()

print("✅ AIProjectClient initialized")
print("✅ OpenAI client ready for conversations")

---

## 🗄️ Step 3: Create a Memory Store

The **Memory Store** is where customer memories are persisted. It uses:
- A **chat model** to extract meaningful memories from conversations
- An **embedding model** to enable semantic search across memories

### Memory Store Options:
| Option | Description |
|--------|-------------|
| `user_profile_enabled` | Extract and maintain user preferences/traits |
| `chat_summary_enabled` | Create summaries of past conversations |

In [ ]:
# Memory store configuration for banking assistant
memory_store_name = "banking-customer-memory-store"

# Delete existing memory store if it exists (for clean demo)
print("🧹 Cleaning up existing memory store (if any)...")
try:
    project_client.memory_stores.delete(memory_store_name)
    print(f"   🗑️ Deleted existing memory store '{memory_store_name}'")
    time.sleep(2)
except ResourceNotFoundError:
    print(f"   ℹ️ No existing memory store found (OK)")

# Define memory store with chat and embedding models
print("\n🏗️ Creating memory store...")

memory_definition = MemoryStoreDefaultDefinition(
    chat_model=memory_chat_model,
    embedding_model=memory_embedding_model,
    options=MemoryStoreDefaultOptions(
        user_profile_enabled=True,   # Extract user preferences
        chat_summary_enabled=True    # Summarize conversations
    )
)

# Create the memory store
memory_store = project_client.memory_stores.create(
    name=memory_store_name,
    description="Memory store for personalized banking assistant - stores customer preferences, risk tolerance, and conversation history",
    definition=memory_definition
)

print("\n" + "=" * 60)
print("✅ Memory Store Created")
print("=" * 60)
print(f"📦 Name: {memory_store.name}")
print(f"🆔 ID: {memory_store.id}")
print(f"📝 Description: {memory_store.description}")
print(f"\n🧠 Memory Capabilities:")
print(f"   • User Profile Extraction: ✅ Enabled")
print(f"   • Chat Summarization: ✅ Enabled")
print(f"   • Semantic Search: ✅ Ready")

---

## 🔧 Step 4: Configure the Memory Search Tool

The **Memory Search Tool** allows the agent to:
1. **Search** for relevant memories during conversations
2. **Update** memories after periods of inactivity

### Key Parameters:
| Parameter | Description |
|-----------|-------------|
| `memory_store_name` | The memory store to search |
| `scope` | User identifier (e.g., customer ID) to isolate memories |
| `update_delay` | Seconds of inactivity before extracting new memories |

In [ ]:
# Define customer scope - in production, this would be the customer's unique ID
# You can also use "{{$userId}}" to automatically use the authenticated user's ID
customer_id = "customer_john_doe_12345"

# Configure Memory Search Tool
memory_tool = MemorySearchTool(
    memory_store_name=memory_store.name,
    scope=customer_id,
    update_delay=5  # Wait 5 seconds of inactivity before updating memories
    # In production, use a higher value like 300 (5 minutes)
)

print("🔧 Memory Search Tool Configured")
print("=" * 60)
print(f"📦 Memory Store: {memory_tool.memory_store_name}")
print(f"👤 Customer Scope: {memory_tool.scope}")
print(f"⏱️ Update Delay: {memory_tool.update_delay} seconds")
print("\n💡 How it works:")
print("   1. Agent automatically searches memories when responding")
print("   2. After inactivity, new memories are extracted and stored")
print("   3. Memories are scoped to this specific customer")

---

## 🤖 Step 5: Create the Memory-Enabled Banking Agent

Create a personalized banking assistant that uses memory to provide contextual, personalized financial guidance.

In [ ]:
# Agent name for the banking assistant
agent_name = "personalized-banking-advisor"

# Detailed instructions for the banking agent
agent_instructions = """
You are a Personalized Banking Advisor with memory capabilities. You remember customer preferences and past interactions to provide tailored financial guidance.

## YOUR CAPABILITIES:

🧠 **Memory-Enabled Personalization**
- You remember customer preferences shared in previous conversations
- You recall risk tolerance, investment preferences, and financial goals
- You reference past discussions to provide continuity

🏦 **Banking Services**
- Savings and checking account guidance
- Certificate of Deposit (CD) recommendations
- Money market account information
- General loan and mortgage information

📊 **Investment Guidance**
- Risk assessment discussions
- Portfolio diversification concepts
- Retirement planning basics (401k, IRA)
- Investment product education

## HOW TO RESPOND:

1. **Use Memory**: Reference what you remember about the customer:
   - "Based on your preference for low-risk investments..."
   - "Since you mentioned you're saving for retirement..."
   - "Given your conservative risk tolerance..."

2. **Ask About Preferences**: If you don't have relevant memories:
   - "What's your risk tolerance for investments?"
   - "What are your primary financial goals?"
   - "Do you prefer aggressive or conservative strategies?"

3. **Personalize Recommendations**: Tailor advice to stored preferences:
   - Match products to risk tolerance
   - Align suggestions with stated goals
   - Reference past discussions when relevant

## IMPORTANT DISCLAIMERS:

⚠️ Always include appropriate disclaimers:
- You are not a licensed financial advisor
- Recommendations are for educational purposes only
- Customers should consult qualified professionals for personalized advice
- Past performance does not guarantee future results

## EXAMPLE INTERACTIONS:

**First Conversation:**
Customer: "I'm looking to invest but I'm risk-averse"
You: Store this preference and suggest conservative options like bonds, CDs, high-yield savings

**Later Conversation:**
Customer: "What should I invest in?"
You: "Based on your conservative risk profile from our previous discussion, you might consider..."
"""

print(f"📝 Creating agent: {agent_name}")
print(f"   Instructions: {len(agent_instructions)} characters")

In [ ]:
# Create the agent with Memory Search Tool
print("🤖 Creating Memory-Enabled Banking Agent...")

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment,
        instructions=agent_instructions,
        tools=[memory_tool]  # Attach the memory search tool
    )
)

print("\n" + "=" * 60)
print("✅ Agent Created with Memory Capabilities")
print("=" * 60)
print(f"🤖 Name: {agent.name}")
print(f"🆔 ID: {agent.id}")
print(f"📌 Version: {agent.version}")
print(f"🧠 Model: {model_deployment}")
print(f"🔧 Tools: Memory Search Tool")
print(f"\n🎯 Agent Capabilities:")
print(f"   • Remember customer preferences across conversations")
print(f"   • Recall past financial discussions")
print(f"   • Provide personalized banking guidance")

---

## 💬 Step 6: Establish Customer Preferences (Conversation 1)

In this first conversation, the customer will share their financial preferences. The agent will:
1. Respond helpfully
2. Store these preferences in memory for future use

In [ ]:
# Helper function to chat with the agent
def chat_with_agent(conversation_id, message, agent_ref):
    """Send a message to the agent and get a response."""
    print(f"\n👤 Customer: {message}")
    print("⏳ Agent thinking...")
    
    response = openai_client.responses.create(
        input=message,
        conversation=conversation_id,
        extra_body={
            "agent": {
                "name": agent_ref.name,
                "version": agent_ref.version,
                "type": "agent_reference"
            }
        }
    )
    
    print(f"\n🤖 Advisor: {response.output_text}")
    return response

print("✅ Chat helper function defined")

In [ ]:
# Create first conversation - establishing preferences
print("📞 Starting Conversation 1: Establishing Customer Preferences")
print("=" * 70)

conversation1 = openai_client.conversations.create()
print(f"✅ Conversation created (ID: {conversation1.id})")

# Customer shares their preferences
preference_messages = [
    "Hi, I'm a new customer. I'm 45 years old and I prefer conservative, low-risk investments. I'm saving for retirement in about 20 years.",
    "I also want to set up an emergency fund. I prefer to keep about 6 months of expenses in liquid savings.",
    "My annual income is around $120,000 and I can invest about $1,500 per month."
]

for msg in preference_messages:
    chat_with_agent(conversation1.id, msg, agent)
    print("\n" + "-" * 70)

print("\n" + "=" * 70)
print("✅ Conversation 1 Complete - Preferences Shared")
print("=" * 70)

---

## ⏳ Step 7: Wait for Memory Extraction

After a period of inactivity (defined by `update_delay`), the agent automatically extracts memories from the conversation and stores them.

**In this demo**: We wait a short time for memories to be extracted.  
**In production**: The `update_delay` would be longer (e.g., 5 minutes) to avoid extracting during active typing.

In [ ]:
# Wait for memories to be extracted
wait_time = 30  # seconds

print("⏳ Waiting for memories to be extracted from Conversation 1...")
print(f"   This simulates the inactivity period ({wait_time} seconds)")
print("")

for i in range(wait_time, 0, -5):
    print(f"   🕐 {i} seconds remaining...")
    time.sleep(5)

print("\n🧠 Memories should now be extracted and stored!")
print("   The agent has learned:")
print("   • Customer is 45 years old")
print("   • Prefers conservative, low-risk investments")
print("   • Planning for retirement in 20 years")
print("   • Wants 6-month emergency fund")
print("   • Income: ~$120,000, can invest $1,500/month")

---

## 🔄 Step 8: Test Memory Recall (Conversation 2)

Now we'll start a **brand new conversation**. The agent should recall the customer's preferences from the memory store and provide personalized responses.

In [ ]:
# Create a NEW conversation - testing memory recall
print("📞 Starting Conversation 2: Testing Memory Recall")
print("=" * 70)
print("🧠 This is a NEW conversation - the agent should remember preferences!")
print("=" * 70)

conversation2 = openai_client.conversations.create()
print(f"✅ New conversation created (ID: {conversation2.id})")

# Ask questions that should trigger memory recall
recall_messages = [
    "Hi, I spoke with someone earlier. Can you recommend some investment options for me?",
    "What about my emergency fund? How should I set that up?",
    "Given my situation, should I max out my 401k contributions?"
]

for msg in recall_messages:
    chat_with_agent(conversation2.id, msg, agent)
    print("\n" + "-" * 70)

print("\n" + "=" * 70)
print("✅ Conversation 2 Complete - Memory Recall Tested")
print("=" * 70)
print("\n🎯 Observe how the agent:")
print("   • Referenced the customer's conservative risk profile")
print("   • Remembered the retirement timeline (~20 years)")
print("   • Recalled the emergency fund preference (6 months)")
print("   • Considered the income and investment capacity")

---

## 🔄 Step 9: Update Preferences (Conversation 3)

Customers' preferences can change over time. Let's test how the agent handles updated preferences.

In [ ]:
# Create conversation to update preferences
print("📞 Starting Conversation 3: Updating Customer Preferences")
print("=" * 70)

conversation3 = openai_client.conversations.create()
print(f"✅ Conversation created (ID: {conversation3.id})")

# Customer updates their preferences
update_messages = [
    "I've been thinking, and I'd like to be a bit more aggressive with my investments. I can handle moderate risk now.",
    "I also got a raise! My income is now $140,000 and I can invest $2,000 per month."
]

for msg in update_messages:
    chat_with_agent(conversation3.id, msg, agent)
    print("\n" + "-" * 70)

print("\n" + "=" * 70)
print("✅ Conversation 3 Complete - Preferences Updated")
print("=" * 70)

# Wait for new memories to be stored
print("\n⏳ Waiting for updated memories to be stored (15 seconds)...")
time.sleep(15)
print("✅ Updated memories should be stored!")

---

## 🧪 Step 10: Verify Updated Memory (Conversation 4)

Let's verify that the agent remembers the **updated** preferences.

In [ ]:
# Create final conversation to verify updated memories
print("📞 Starting Conversation 4: Verifying Updated Preferences")
print("=" * 70)
print("🧠 The agent should now remember the UPDATED preferences!")
print("=" * 70)

conversation4 = openai_client.conversations.create()
print(f"✅ Conversation created (ID: {conversation4.id})")

# Ask about investment recommendations with updated profile
verify_message = "Based on everything you know about me, what's your top investment recommendation right now?"

chat_with_agent(conversation4.id, verify_message, agent)

print("\n" + "=" * 70)
print("✅ Memory Verification Complete")
print("=" * 70)
print("\n🎯 The agent should have referenced:")
print("   • Updated risk tolerance: moderate (not conservative)")
print("   • Updated income: $140,000 (not $120,000)")
print("   • Updated investment capacity: $2,000/month (not $1,500)")

---

## 🧹 Step 11: Cleanup Resources

Delete all created resources to clean up.

In [ ]:
# Cleanup resources
print("🧹 Cleaning up resources...")
print("=" * 60)

# Delete conversations
conversations_to_delete = [conversation1, conversation2, conversation3, conversation4]
for i, conv in enumerate(conversations_to_delete, 1):
    try:
        openai_client.conversations.delete(conv.id)
        print(f"✅ Deleted conversation {i}: {conv.id}")
    except Exception as e:
        print(f"⚠️ Could not delete conversation {i}: {e}")

# Delete agent
try:
    project_client.agents.delete_version(
        agent_name=agent.name,
        agent_version=agent.version
    )
    print(f"✅ Deleted agent: {agent.name} (v{agent.version})")
except Exception as e:
    print(f"⚠️ Could not delete agent: {e}")

# Delete memory store
try:
    project_client.memory_stores.delete(memory_store.name)
    print(f"✅ Deleted memory store: {memory_store.name}")
except Exception as e:
    print(f"⚠️ Could not delete memory store: {e}")

print("\n" + "=" * 60)
print("🎉 Cleanup complete!")

---

## 📚 Summary

In this notebook, you learned how to build a **memory-enabled banking assistant** that provides personalized financial guidance by remembering customer preferences.

### What You Built

| Component | Description |
|-----------|-------------|
| **Memory Store** | Persistent storage for customer preferences and conversation summaries |
| **Memory Search Tool** | Agent tool that retrieves and updates memories |
| **Banking Agent** | AI assistant that uses memories for personalization |

### Key Concepts Demonstrated

1. **Memory Extraction** - Automatically extract preferences from conversations
2. **Memory Recall** - Retrieve relevant memories in new conversations
3. **Memory Updates** - Handle changing customer preferences over time
4. **Scoped Memories** - Isolate memories per customer for privacy

### Industry Use Cases for Agent Memory

| Use Case | Memory Benefit |
|----------|----------------|
| **Wealth Management** | Remember risk tolerance, investment preferences, financial goals |
| **Customer Service** | Recall past issues, preferred communication channels, account history |
| **Loan Processing** | Remember document submissions, employment history, property details |
| **Insurance** | Recall policy preferences, claims history, coverage requirements |
| **Fraud Detection** | Remember customer behavior patterns, typical transaction types |

### Production Considerations

| Consideration | Recommendation |
|---------------|----------------|
| **Update Delay** | Set to 300+ seconds (5 min) to avoid extraction during active conversations |
| **Scope Strategy** | Use customer IDs or `{{$userId}}` for automatic user isolation |
| **Memory Retention** | Consider data retention policies and GDPR compliance |
| **Sensitive Data** | Avoid storing PII or sensitive financial data in memories |

---

**🎉 Congratulations!** You've successfully built a personalized banking assistant with memory capabilities!

*Note: This is for educational purposes. Production deployments require proper error handling, security, and compliance features.*